<a href="https://colab.research.google.com/github/carmen-chan/A2/blob/master/Testing_area_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
url = 'https://raw.githubusercontent.com/carmen-chan/A2/master/heart.csv'
df = pd.read_csv(url, error_bad_lines=False)

list(df.columns.values)

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target']